In [ ]:
from dwave.system import DWaveSampler
import minorminer as mm
import dwave.inspector

In [ ]:
f = open("token.txt", "r")
sampler = DWaveSampler(solver=dict(topology__type='pegasus'),token=f.read())
print("QPU {} was selected.".format(sampler.solver.name))

In [ ]:
#create a target graph
T=nx.Graph()
T.add_edges_from(sampler.edgelist)
T.add_nodes_from(sampler.nodelist)

In [ ]:
#create a source graph
S=nx.Graph()
#spin chain length
N=2000
#define the nodes
S.add_nodes_from(list(range(N)))
spins=list(S.nodes)
#define the edges. the first node is also the last node (PBC)
for i in range(len(spins)-1):
    S.add_edge(spins[i],spins[i+1])
S.add_edge(spins[len(spins)-1],spins[0])

In [ ]:
cond=0
count=0

while (cond==0 and count<1000):

    embedding=mm.find_embedding(S,T,verbose=1)

    biggerthan1=[]
    for value in embedding.values():
        if len(value)>1:
            biggerthan1.append(value)
    
    if len(biggerthan1)>0:
        cond=0
    if len(biggerthan1)==0:
        cond=1
        
    if count==1000:
        print("reached max interations")
        
    count+=1

# print(embedding)

In [ ]:
couplers={}
for edge in list(S.edges):
    spin0=edge[0]
    spin1=edge[1]
    qubit0=embedding.get(spin0)[0]
    qubit1=embedding.get(spin1)[0]
    couplers.update({(qubit0,qubit1):-1})